In [1]:
#--------------------------------------------------------------------------------------------------- 

import numpy as np
from scipy.integrate import quad
from scipy import optimize
import math


class cosmology:
    def __init__(self, Omega_m, Omega_r, Omega_l, H0):
        self.Omega_m = Omega_m
        self.Omega_r = Omega_r   
        self.Omega_l = Omega_l
        self.H0 = H0

        # Define the curvature based on the Friedmann equation evaluated at t=today.
        self.Omega_k = 1 - Omega_m - Omega_r - Omega_l

        self.a_max = self.max_scale_factor()
        print(self.a_max, "Maximum scale factor")

    def zofa(self, a):
        return 1./a - 1.
    #redshift to scale parameter
    def aofz(self, z):
        return 1/(1.+z)

    # Define the RHS of the Friedmann equation
    def Esqofa(self, a):
        return (self.Omega_m*a**-3 + self.Omega_r*a**-4 + self.Omega_l + self.Omega_k*a**-2 )

    # Define derivative of the RHS of the Friedmann equation
    def Esqprimeofa(self, a):
        return (-3.*self.Omega_m*a**-4 - 4.*self.Omega_r*a**-5 - 2.*self.Omega_k*a**-3 )

    # Define the RHS of the Friedmann equation
    def Eofa(self, a):
        return (self.Omega_m*a**-3 + self.Omega_r*a**-4 + self.Omega_l + self.Omega_k*a**-2 )**0.5
        

    # Define derivative of the RHS of the Friedmann equation
    def Eprimeofa(self, a):
        return (-3*self.Omega_m*a**-4 - 4*self.Omega_r*a**-5 - 2*self.Omega_k*a**-3 )/(self.Omega_m*a**-3 + self.Omega_r*a**-4 + self.Omega_l + self.Omega_k*a**-2 )**0.5


    # Define the time given a scale factor (time will be returned in units of 1/H0)
    def compute_time_given_scale(self, a):
        # Integrate the function \int da/a/E(a) = \int d(t/(1/H0))
        def integrand(ap):
            return 1/ap/self.Eofa(ap)
        if a>self.a_max:
            return np.nan
        #answer=0
        #print("ans")
        answer, error = quad(integrand, 1, a)
        return answer

    # Compute maximum scale factor if the Universe is closed based on the zero of Eofa
    def max_scale_factor(self):
        if self.Omega_k>=0:
            return np.inf

        # Compute the zero of the expansion factor
        sol = optimize.root_scalar(self.Esqofa, x0=1.0, fprime=self.Esqprimeofa, method='newton')

        if sol.flag == "convergence error":
            sol.root = np.inf

        return sol.root
               #print(scale_arr)
        
#--------------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------------
    # chi = integration of [c da / a**2(adot/a)] from a to 1 
    def chi(self, a):
        def integrand(ap):
            return ((3e5)/((ap**2)*self.H0*self.Eofa(ap))) 
        
        if a>self.a_max:
            return np.nan
        x = 0
        x, error = quad(integrand, a, 1)
        return x 

 #---------------------------------------------------------------------------------------------------------   
    def fun_chi(self, a):
        k= -(( self.Omega_k*(self.H0**2))/3e5)
        #print("k=", k)
        if k>0:
            R=math.sqrt(1.0/k)
            #print("universe = positive curved")
            f= R*math.sin(self.chi(a)/R)
        elif k==0:
         #   print("universe= flat")
            f= self.chi(a)
        else:
            print("universe = negative curved")
            R=math.sqrt(-1/k)
            f= R*(math.sinh(self.chi(a)/R))
        #print("fun(chi)=", f)
        return f
#--------------------------------------------------------------------------------------------------------------    
    #Both angular diameter distance and luminosity distance
    #Dangular=f(chi)*a
    #Dlum=f(chi)/a
    def angularandlum(self, a):
        angular= a*self.fun_chi(a)
        luminosity = angular*((1/a)**2) 
        print("angular diemeter distance=",angular,"mpc     " "luminosity distance=",luminosity,"mpc")
        return angular,luminosity
#--------------------------------------------------------------------------------------------------------------
    
        #horizon distance in Mpc
    def horizon(self, a):
        def hori(ap):
            return (3e5)/((ap**2)*self.H0*self.Eofa(ap)) 
        if a>self.a_max:
            return np.nan
        hulk = 0
        hulk, error = quad(hori, 0, 1)
        return hulk
#----------------------------------------------------------------------------------------------------------------    
   #age of universe for given scale factor  in year (taking scale factor a=1 for today)
    def age(self, a):
        H00=self.H0/3.08e19 # coverting from km/s/Mpc to 1/sec
        t=self.compute_time_given_scale(a)/H00    #(t = 1/H0)
        t1=t/(60*60*24*365)                       # seconds to year 
        t2=(13786758138.658695)+t1                #adjusting y axis 
        return t2
#---------------------------------------------------------------------------------------------------------------    
 
    #parsec to lightyear 
    def parsec_to_lightyear(self,p):
        return p*3.26156
    def ly_to_meter(self,ly):
        return ly*9.461e+15
#---------------------------------------------------------------------------------------------------------------    
    #distance to redshift (put distance in Mpc)  ---- cosmological redshift ---(for dist << c/H0) 
    def dist_to_redshift(self, p): 
        return ((self.H0*p)/ 3e5)
    
#---------------------------------------------------------------------------------------------------------------    
    
    #angular diemeter distance in unit of Mpc
    def angular(self, a):
        angular= a*self.fun_chi(a)
        return angular
   #luminosity distance in unit Mpc 
    def lum(self, a):
        luminosity = self.angular(a)*((1/a)**2) 
        return luminosity
    
    #angular size to linear size
    #theta = "theta(arcminute) :"))         
    #Dc = "Dc(Mpc) :"))
    #z = (redshift :"))
    def size(self,theta,Dc,z): 
        lin_size = (theta*Dc*np.pi)/((1+z)*10800)
        return print(lin_size,"Mpc")


In [11]:
multiverse= cosmology(0.3089,0,0.6911,67.74)

inf Maximum scale factor


In [12]:
scale_arr = np.linspace(1e-2, 3., 10000)
print(scale_arr)

#time_arr_flat = scale_arr*0.0
arr_angular = scale_arr*0.0
arr_lum = scale_arr*0.0
time_arr_open = scale_arr*0.0
for ii in range(scale_arr.size):
    #time_arr_flat[ii] = multiverse.compute_time_given_scale(scale_arr[ii])
    arr_angular[ii] = multiverse.angular(scale_arr[ii])
    arr_lum[ii] = multiverse.lum(scale_arr[ii])
    time_arr_open[ii] = multiverse.compute_time_given_scale(scale_arr[ii])

[0.01       0.01029903 0.01059806 ... 2.99940194 2.99970097 3.        ]


In [13]:

import pylab as pl
%pylab inline

conf = %config InlineBackend.rc
conf["figure.figsize"] = (6, 6)
conf['savefig.dpi']=100
conf['font.serif'] = "Computer Modern"
conf['font.sans-serif'] = "Computer Modern"
conf['text.usetex']=True

width = 600
%config InlineBackend.rc
%pylab inline


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [ ]:

ax = pl.subplot(111)
idx = ~(time_arr_open==0)
ax.plot(scale_arr[idx],arr_angular[idx], label="angular diameter distance ")
ax.plot(scale_arr[idx],arr_lum[idx], label="luminosity distsnce")
ax.set_xlabel(r"scale factor a")
ax.set_ylabel("Angular diameter and Luminosity distance(Mpc)")
ax.axvline(0.0, color="k", alpha=0.6)
ax.legend()
ax.set_xlim(0.0, 3)
ax.set_ylim(-7000, 7000)
ax.text(0.96,-7000,"--------------today------------------------------------------------------------------------------", rotation=90)


In [ ]:
multiverse = cosmology(0.3,0,0.7,100)

In [ ]:

scale_arr = np.linspace(1e-2, 300, 10000)
arr_time = scale_arr*0.0
for ii in range(scale_arr.size):
    arr_time[ii] = multiverse.compute_time_given_scale(scale_arr[ii])

In [ ]:

ax = pl.subplot(111)

idx = ~(time_arr_closed==0)
ax.plot(arr_time[idx],scale_arr[idx], label="scale factor a ")
ax.plot(scale_arr[idx],arr_lum[idx], label="lum")
ax.set_xlabel(r"time (1/H0)")
ax.set_ylabel("scale  factor a")
ax.axvline(0.0, color="k", alpha=0.6)
ax.legend()
ax.set_xlim(-2, 10)
ax.set_ylim(0, 30)
ax.text(-0.5, 15, "today", rotation=90)


In [ ]:

scale_arr = np.linspace(1e-2, 300, 10000)
arr_time = scale_arr*0.0
for ii in range(scale_arr.size):
    arr_time[ii] = rahul.age(scale_arr[ii])

In [ ]:

ax = pl.subplot(111)

idx = ~(time_arr_closed==0)
ax.plot(scale_arr[idx],arr_time[idx], label="age")
ax.set_xlabel(r"scale factor a")
ax.set_ylabel("age in year")
ax.axvline(0.0, color="k", alpha=0.6)
ax.legend()
ax.set_xlim(0, 5)
ax.set_ylim(0,60000000000)
ax.text(0.94, 0, "----------------------------------------------------today---------------------------------------", rotation=90)
ax.text(0, 13786758138.658695, "----------------------------------------------------13.78 Gyr--------------------------------------", rotation=0)
